In [1]:
import sys
import numpy as np
import time
import os
import logging
from collections import Counter
from datetime import datetime
import math

from tqdm import tqdm
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim

To make the dependency parser, there will be lot of component : 

The transition parsing function,
Loading the data (conll),
The actual parser,
The embedding lookup,
The deep learning model,
The training and evaluation

Parsing Function. We will use class Prasing.

In [2]:
class Parsing(object):
    def __init__(self, sentence):
        self.sentence = sentence #['The', 'cat', 'sat']
        
        # The current stack represented as a list with the top of the stack as the
        # last element of the list.
        self.stack = ['ROOT']
        
        # The current buffer represented as a list with the first item on the
        # buffer as the first item of the list
        self.buffer = sentence[:]
        
        # The list of dependencies produced so far. Represented as a list of
        # tuples where each tuple is of the form (head, dependent).
        self.dep = []
    
    #Let's first create a function to perform "Shift", "Left-Arc", and "Right-Arc"
    def parse_step(self, transition):
        if transition == 'S':
            buffer_head = self.buffer.pop(0)
            self.stack.append(buffer_head)
        #stack: [ROOT, I, parsed]
        elif transition == 'LA':
            dependent = self.stack.pop(-2) #I
            self.dep.append((self.stack[-1], dependent)) #(parsed, I)
        #stack: [ROOT, parsed, sentence]
        elif transition == 'RA':
            dependent = self.stack.pop() #sentence
            self.dep.append((self.stack[-1], dependent)) #(parsed, sentence)
        else:
          print(f"Unknown transition: {transition}")
            
    #loop all transitions
    def parse(self, transitions):
        for transition in transitions:
            self.parse_step(transition)
        return self.dep
    
    #an utility function to check that the parsing is done.
    def is_completed(self):
        return (len(self.buffer) == 0) and (len(self.stack) == 1)

Testing the parsing step

In [3]:
parsing = Parsing(["The", "cat", "sat"])
parsing.stack, parsing.buffer, parsing.dep

(['ROOT'], ['The', 'cat', 'sat'], [])

In [4]:
parsing.parse_step("S")
parsing.stack, parsing.buffer, parsing.dep

(['ROOT', 'The'], ['cat', 'sat'], [])

In [5]:
parsing.parse_step("LA")
parsing.stack, parsing.buffer, parsing.dep

(['The'], ['cat', 'sat'], [('The', 'ROOT')])

In [6]:
parsing.parse_step("S")
parsing.stack, parsing.buffer, parsing.dep

(['The', 'cat'], ['sat'], [('The', 'ROOT')])

In [7]:
parsing.parse_step("RA")
parsing.stack, parsing.buffer, parsing.dep

(['The'], ['sat'], [('The', 'ROOT'), ('The', 'cat')])

Testing the parse

In [8]:
parsing = Parsing(["The", "cat", "sat"])
parsing.parse(["S", "LA", "S", "RA"])
parsing.stack, parsing.buffer, parsing.dep

(['The'], ['sat'], [('The', 'ROOT'), ('The', 'cat')])

We gonna create a minibatch loader that loads a bunch of sentences, and perform parse accordingly

In [9]:
def minibatch_parse(sentences, model, batch_size):
    dep = []

    # Initialize a list of DepParser, one for each sentence.
    partial_parses = [Parsing(sentence) for sentence in sentences]
    
    # Shallow copy partial_parses.
    unfinished_parses = partial_parses[:]
        
    # While unfinished_parses is not empty do.
    while unfinished_parses:
        # Take first batch_size parses in unfinished_parses as minibatch.
        minibatch = unfinished_parses[:batch_size]
        # Use model to predict next transition for each partial parse in the minibatch.
        transitions = model.predict(minibatch)
        # Perform parse step for each partial_parse_minibatch with their predicted transition.
        for transition, partial_parse in zip(transitions, minibatch):
            partial_parse.parse_step(transition)
        # Remove completed parses.
        unfinished_parses[:] = [
            p for p in unfinished_parses if not p.is_completed()]
    dep = [parse.dep for parse in partial_parses]

    return dep

In [12]:
class DummyModel(object):
    """
    Dummy model for testing the minibatch_parse function
    First shifts everything onto the stack and then does exclusively right arcs if the first word of
    the sentence is "right", "left" if otherwise.
    """

    def predict(self, partial_parses):
        return [("RA" if pp.stack[1] == "right" else "LA") if len(pp.buffer) == 0 else "S"
                for pp in partial_parses]

In [13]:
sentences = [["right", "arcs", "only"],
             ["right", "arcs", "only", "again"],
             ["left", "arcs", "only"],
             ["left", "arcs", "only", "again"]]
dep = minibatch_parse(sentences, DummyModel(), 2)

In [14]:
dep

[[('arcs', 'only'), ('right', 'arcs'), ('ROOT', 'right')],
 [('only', 'again'), ('arcs', 'only'), ('right', 'arcs'), ('ROOT', 'right')],
 [('only', 'arcs'), ('only', 'left'), ('only', 'ROOT')],
 [('again', 'only'), ('again', 'arcs'), ('again', 'left'), ('again', 'ROOT')]]

Load dataset

In [29]:
from google.colab import drive

In [30]:
drive.mount('/gdrive')

Mounted at /gdrive


In [31]:
class Config(object):
    data_path = '/gdrive/MyDrive'
    train_file = '/gdrive/MyDrive/train.conll'
    dev_file = '/gdrive/MyDrive/dev.conll'
    test_file = '/gdrive/MyDrive/test.conll'
    embedding_file = '/gdrive/MyDrive'

In [32]:
def read_conll(filename, max_example=5):
    examples = []
    with open(filename) as f:
        i = 0
        word, pos, head, dep = [], [], [], []
        for line in f.readlines():
            i = i + 1
            wa = line.strip().split('\t') #wa = word annotations
            
            #if all 10 columns are there....
            if len(wa) == 10:  
                word.append(wa[1].lower())
                pos.append(wa[4])
                head.append(int(wa[6]))
                dep.append(wa[7])
            
            #otherwise, it means the line is empty, thus we consider finish for one example
            elif len(word) > 0:
                examples.append({'word': word, 'pos': pos, 'head': head, 'dep': dep})
                word, pos, head, dep = [], [], [], []
                    
        #last example
        if len(word) > 0:
            examples.append({'word': word, 'pos': pos, 'head': head, 'dep': dep})
            
    return examples

In [33]:
import time

#sample_size = (train, dev, test)
def load_data(sample_size=(1000, 500, 500)):
    config = Config()

    print("1. Loading data...",)
    start = time.time()
    train_set = read_conll(os.path.join(config.data_path, config.train_file))
    dev_set   = read_conll(os.path.join(config.data_path, config.dev_file))
    test_set  = read_conll(os.path.join(config.data_path, config.test_file))

    #for shorter preprocessing time.....just for education, ok guys?
    train_set = train_set[:sample_size[0]]
    dev_set   = dev_set[:sample_size[1]]
    test_set  = test_set[:sample_size[2]]
    print("Example 1: ", train_set[1]) #i choose one because it's short

    print("took {:.2f} seconds".format(time.time() - start))
    
    return train_set, dev_set, test_set

In [34]:
print(80 * "=")
print("INITIALIZING")
print(80 * "=")
train_set, dev_set, test_set = load_data()

INITIALIZING
1. Loading data...
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 4.07 seconds


In [35]:
P_PREFIX = '<p>:' #indicating pos tags
D_PREFIX = '<d>:' #indicating dependency tags
UNK      = '<UNK>'
NULL     = '<NULL>'
ROOT     = '<ROOT>'

class Parser(object):

    def __init__(self, dataset):
        
        #set the root dep
        self.root_dep = 'root'
                
        #get all the dep of the dataset as list, e.g., ['root', 'acl', 'nmod', 'nmod:npmod']
        all_dep = [self.root_dep] + list(set([w for ex in dataset
                                               for w in ex['dep']
                                               if w != self.root_dep]))
        
        #1. put dep into tok2id lookup table, with D_PREFIX so we know it is dependency
        #{'D_PREFIX:root': 0, 'D_PREFIX:acl': 1, 'D_PREFIX:nmod': 2, ..., 'D_PREFIX:<NULL>': 30}
        tok2id = {D_PREFIX + l: i for (i, l) in enumerate(all_dep)}
        tok2id[D_PREFIX + NULL] = self.D_NULL = len(tok2id)
        
        #we are using "unlabeled" where we do not label with the dependency
        #thus the number of dependency relation is 1
        trans = ['L', 'R', 'S']
        self.n_deprel = 1
        
        #create a simple lookup table mapping action and id
        #e.g., tran2id: {'L': 0, 'R': 1, 'S': 2}
        #e.g., id2tran: {0: 'L', 1: 'R', 2: 'S'}
        self.n_trans = len(trans)
        self.tran2id = {t: i for (i, t) in enumerate(trans)}
        self.id2tran = {i: t for (i, t) in enumerate(trans)}

        #2. put pos tags into tok2id lookup table, with P_PREFIX so we know it is pos
        tok2id.update(build_dict([P_PREFIX + w for ex in dataset for w in ex['pos']],
                                  offset=len(tok2id)))
        tok2id[P_PREFIX + UNK]  = self.P_UNK  = len(tok2id)  #also remember the pos tags of unknown
        tok2id[P_PREFIX + NULL] = self.P_NULL = len(tok2id)
        tok2id[P_PREFIX + ROOT] = self.P_ROOT = len(tok2id)
        
        #now tok2id:  {'P_PREFIX:root': 0, 'P_PREFIX:acl': 1, ..., 'P_PREFIX:JJR': 62, 'P_PREFIX:<UNK>': 63, 'P_PREFIX:<NULL>': 64, 'P_PREFIX:<ROOT>': 65}
        
        #3. put word into tok2id lookup table
        tok2id.update(build_dict([w for ex in dataset for w in ex['word']],
                                  offset=len(tok2id)))
        tok2id[UNK]  = self.UNK = len(tok2id)
        tok2id[NULL] = self.NULL = len(tok2id)
        tok2id[ROOT] = self.ROOT = len(tok2id)
        
        #now tok2id: {'D_PREFIX:root': 0, 'D_PREFIX:acl': 1, 'D_PREFIX:nmod': 2, ..., 'memory': 340, 'mr.': 341, '<UNK>': 342, '<NULL>': 343, '<ROOT>': 344}
        
        #create id2tok
        self.tok2id = tok2id
        self.id2tok = {v: k for (k, v) in tok2id.items()}
        
        #why 18 normal features + 18 (pos) + 12 (dep)
        #18 features - top 3 words on buffer, top 3 words on stack, 
        # the first and second left most/rightmost children of the top two words on the stack
        # the leftmost of leftmost/rightmost of rightmost children of the top two words on the stack
        #18 pos - basically corresponding POS tags
        #12 dep - corresponding ARC, excluding 6 words on hte stack/buffer..
        self.n_features = 18 + 18 + 12
        self.n_tokens = len(tok2id)
    
    #function to turn train set with words to train set with id instead using tok2id
    def numericalize(self, examples):
        numer_examples = []
        for ex in examples:
            word = [self.ROOT] + [self.tok2id[w] if w in self.tok2id
                                  else self.UNK for w in ex['word']]
            pos  = [self.P_ROOT] + [self.tok2id[P_PREFIX + w] if P_PREFIX + w in self.tok2id
                                   else self.P_UNK for w in ex['pos']]
            head = [-1] + ex['head']
            dep  = [-1] + [self.tok2id[D_PREFIX + w] if D_PREFIX + w in self.tok2id
                            else -1 for w in ex['dep']]
            numer_examples.append({'word': word, 'pos': pos,
                                 'head': head, 'dep': dep})
        return numer_examples

    #function to extract features to form a feature embedding matrix
    def extract_features(self, stack, buf, arcs, ex):
             
        #ex['word']:  [55, 32, 33, 34, 35, 30], i.e., ['root', 'ms.', 'haag', 'plays', 'elianti', '.']
        #ex['pos']:   [29, 14, 14, 16, 14, 17], i.e., ['NNP', 'NNP', 'VBZ', 'NNP', '.']
        #ex['head']:  [-1, 2, 3, 0, 3, 3]  or ['root', 'compound', 'nsubj', 'root', 'dobj', 'punct']}
        #ex['dep']:   [-1, 1, 2, 0, 6, 12] or ['compound', 'nsubj', 'root', 'dobj', 'punct']

        #stack     :  [0]
        #buffer    :  [1, 2, 3, 4, 5]
        
        if stack[0] == "ROOT":
            stack[0] = 0  #start the stack with [ROOT]

        #get leftmost children based on the dependency arcs
        def get_lc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] < k])

        #get right most children based on the dependency arcs
        def get_rc(k):
            return sorted([arc[1] for arc in arcs if arc[0] == k and arc[1] > k],
                          reverse=True)

        p_features = [] #pos features (2a, 2b, 2c) - 18
        d_features = [] #dep features (3b, 3c) - 12
        
        #last 3 things on the stack as features
        #if the stack is less than 3, then we simply append NULL from the left
        features = [self.NULL] * (3 - len(stack)) + [ex['word'][x] for x in stack[-3:]]
        
        # next 3 things on the buffer as features
        #if the buffer is less than 3, simply append NULL
        #the reason why NULL is appended on end because buffer is read left to right
        features += [ex['word'][x] for x in buf[:3]] + [self.NULL] * (3 - len(buf))
        
        #corresponding pos tags
        p_features = [self.P_NULL] * (3 - len(stack)) + [ex['pos'][x] for x in stack[-3:]]
        p_features += [ex['pos'][x] for x in buf[:3]] + [self.P_NULL] * (3 - len(buf))
        
        #get the leftmost and rightmost children of the top two words, thus we loop 2 times
        for i in range(2):
            if i < len(stack):
                k = stack[-i-1] #-1, -2 last two in the stack
                
                #the first and second lefmost/rightmost children of the top two words (i=1, 2) on the stack
                lc = get_lc(k)  
                rc = get_rc(k)
                
                #the leftmost of leftmost/rightmost of rightmost children of the top two words on the stack:
                llc = get_lc(lc[0]) if len(lc) > 0 else []
                rrc = get_rc(rc[0]) if len(rc) > 0 else []

                #(leftmost of first word on stack, rightmost of first word, 
                # leftmost of the second word on stack, rightmost of second, 
                # leftmost of leftmost, rightmost of rightmost
                features.append(ex['word'][lc[0]] if len(lc) > 0 else self.NULL)
                features.append(ex['word'][rc[0]] if len(rc) > 0 else self.NULL)
                features.append(ex['word'][lc[1]] if len(lc) > 1 else self.NULL)
                features.append(ex['word'][rc[1]] if len(rc) > 1 else self.NULL)
                features.append(ex['word'][llc[0]] if len(llc) > 0 else self.NULL)
                features.append(ex['word'][rrc[0]] if len(rrc) > 0 else self.NULL)

                #corresponding pos
                p_features.append(ex['pos'][lc[0]] if len(lc) > 0 else self.P_NULL)
                p_features.append(ex['pos'][rc[0]] if len(rc) > 0 else self.P_NULL)
                p_features.append(ex['pos'][lc[1]] if len(lc) > 1 else self.P_NULL)
                p_features.append(ex['pos'][rc[1]] if len(rc) > 1 else self.P_NULL)
                p_features.append(ex['pos'][llc[0]] if len(llc) > 0 else self.P_NULL)
                p_features.append(ex['pos'][rrc[0]] if len(rrc) > 0 else self.P_NULL)
            
                #corresponding dep
                d_features.append(ex['dep'][lc[0]] if len(lc) > 0 else self.D_NULL)
                d_features.append(ex['dep'][rc[0]] if len(rc) > 0 else self.D_NULL)
                d_features.append(ex['dep'][lc[1]] if len(lc) > 1 else self.D_NULL)
                d_features.append(ex['dep'][rc[1]] if len(rc) > 1 else self.D_NULL)
                d_features.append(ex['dep'][llc[0]] if len(llc) > 0 else self.D_NULL)
                d_features.append(ex['dep'][rrc[0]] if len(rrc) > 0 else self.D_NULL)
                
            else:
                #attach NULL when they don't exist
                features += [self.NULL] * 6
                p_features += [self.P_NULL] * 6
                d_features += [self.D_NULL] * 6

        features += p_features + d_features
        assert len(features) == self.n_features  #assert they are 18 + 18 + 12
        return features

    #decide whether to shift, leftarc, or rightarc, based on gold parse trees
    #this is needed to create training examples which contain samples and ground truth
    def get_oracle(self, stack, buf, ex):

        #leave if the stack is only 1, thus nothing to predict....
        if len(stack) < 2:
            return self.n_trans - 1

        #predict based on the last two words on the stack
        i0 = stack[-1]
        i1 = stack[-2]

        #get the head and dependency
        h0 = ex['head'][i0]
        h1 = ex['head'][i1]
        d0 = ex['dep'][i0]
        d1 = ex['dep'][i1]

        #either shift, left arc or right arc
        #"Shift" = 2; "LA" = 0; "RA" = 1
        #if head of the second last word is the last word, then leftarc
        if (i1 > 0) and (h1 == i0):
            return 0
        #if head of the last word is the second last word, then rightarc
        #make sure nothing in the buffer has head with the last word on the stack
        #otherwise, we lose the last word.....
        elif (i1 >= 0) and (h0 == i1) and \
                (not any([x for x in buf if ex['head'][x] == i0])):
            return 1
        #otherwise shift, if something is left in buffer, otherwise, do nothing....
        else:
            return None if len(buf) == 0 else 2

    #generate training examples
    #from the training sentences and their gold parse trees 
    def create_instances(self, examples):
        all_instances = []
        
        for i, ex in enumerate(examples):
            #e.g., ex['word]: [344, 163, 99, 164, 165, 68]
            n_words = len(ex['word']) - 1  #excluding the root

            #arcs = {(head, tail, dependency label)}
            stack = [0]
            buf = [i + 1 for i in range(n_words)]  #[1, 2, 3, 4, 5]
            arcs = []
            instances = []
            
            #because that's the maximum number of shift, leftarcs, rightarcs you can have
            #this will determine the sample size of each training example
            #if given five words, we will get a sample of (10, 48) where 10 comes from 5 * 2, and 48 is n_features
            #but this for loop can be break if there is nothing left....
            for i in range(n_words * 2):

                #get the gold transition based on the parse trees
                #gold_t can be either shift(2), leftarc(0), or rightarc(1)
                gold_t = self.get_oracle(stack, buf, ex)
                
                #if gold_t is None, no need to extract features.....
                if gold_t is None:
                    break
                
                #make sure when the model predicts, we inform the current state of stack and buffer, so
                #the model is not allowed to make any illegal action, e.g., buffer is empty but trying to pop
                legal_labels = self.legal_labels(stack, buf)                
                assert legal_labels[gold_t] == 1
               
                #extract all the 48 features 
                features = self.extract_features(stack, buf, arcs, ex)
                instances.append((features, legal_labels, gold_t))
            
                #shift 
                if gold_t == 2:
                    stack.append(buf[0])
                    buf = buf[1:]
                #left arc 
                elif gold_t == 0:
                    arcs.append((stack[-1], stack[-2], gold_t))
                    stack = stack[:-2] + [stack[-1]]
                #right arc
                else:
                    arcs.append((stack[-2], stack[-1], gold_t - self.n_deprel))
                    stack = stack[:-1]
            else:
                all_instances += instances

        return all_instances

    #provide an one hot encoding of the labels
    def legal_labels(self, stack, buf):
        labels =  ([1] if len(stack) > 2  else [0]) * self.n_deprel   #left arc   But cannot ROOT <----He thus 3
        labels += ([1] if len(stack) >= 2 else [0]) * self.n_deprel   #right arc  ROOT--->He
        labels += [1] if len(buf) > 0 else [0]   #shift
        return labels
    
    #a simple function to check punctuation POS tags
    def punct(self, pos):
        return pos in ["''", ",", ".", ":", "``", "-LRB-", "-RRB-"]

    def parse(self, dataset, eval_batch_size=5000):
        sentences = []
        sentence_id_to_idx = {}
                
        for i, example in enumerate(dataset):
            
            #example['word']=[188, 186, 186, ..., 59]
            #n_words=37
            #sentence=[1, 2, 3, 4, 5,.., 37]
                        
            n_words = len(example['word']) - 1
            sentence = [j + 1 for j in range(n_words)]            
            sentences.append(sentence)
            
            #mapping the object unique id to the i            
            #The id is the object's memory address
            sentence_id_to_idx[id(sentence)] = i
            
        model = ModelWrapper(self, dataset, sentence_id_to_idx)
        dependencies = minibatch_parse(sentences, model, eval_batch_size)
                
        UAS = all_tokens = 0.0
        with tqdm(total=len(dataset)) as prog:
            for i, ex in enumerate(dataset):
                head = [-1] * len(ex['word'])
                for h, t, in dependencies[i]:
                    head[t] = h
                for pred_h, gold_h, gold_l, pos in \
                        zip(head[1:], ex['head'][1:], ex['dep'][1:], ex['pos'][1:]):
                        assert self.id2tok[pos].startswith(P_PREFIX)
                        pos_str = self.id2tok[pos][len(P_PREFIX):]
                        if (not self.punct(pos_str)):
                            UAS += 1 if pred_h == gold_h else 0
                            all_tokens += 1
                prog.update(i + 1)
        UAS /= all_tokens
        return UAS, dependencies

In [36]:
def build_dict(keys, offset=0):
    #keys = ['P_PREFIX:IN', 'P_PREFIX:DT', 'P_PREFIX:NNP', 'P_PREFIX:CD', so on...]
    #offset is needed because this tok2id has something already inside....
    count = Counter()
    for key in keys:
        count[key] += 1
    
    #most_common = [('P_PREFIX:NN', 70), ('P_PREFIX:IN', 57), ... , ('P_PREFIX:JJR', 1)]
    #we use most_common in case we only want some maximum pos tags....
    mc = count.most_common()
    
    #{'P_PREFIX:NN': 31, 'P_PREFIX:IN': 32, .., 'P_PREFIX:JJR': 62} 
    return {w[0]: index + offset for (index, w) in enumerate(mc)}

In [64]:
print("4. Loading pretrained embeddings...",)
config = Config()
start = time.time()
word_vectors = {}
for line in open(config.embedding_file).readlines():
    we = line.strip().split() #we = word embeddings - first column: word;  the rest is embedding
    word_vectors[we[0]] = [float(x) for x in we[1:]] #{word: [list of 50 numbers], nextword: [another list], so on...}
    
#create an empty embedding matrix holding the embedding lookup table (vocab size, embed dim)
#we use random.normal instead of zeros, to keep the embedding matrix arbitrary in case word vectors don't exist....
embeddings_matrix = np.asarray(np.random.normal(0, 0.9, (parser.n_tokens, 50)), dtype='float32')

for token in parser.tok2id:
        i = parser.tok2id[token]
        if token in word_vectors:
            embeddings_matrix[i] = word_vectors[token]
        elif token.lower() in word_vectors:
            embeddings_matrix[i] = word_vectors[token.lower()]
print("Embedding matrix shape (vocab, emb size): ", embeddings_matrix.shape)
print("took {:.2f} seconds".format(time.time() - start))

In [37]:
def get_minibatches(data, minibatch_size, shuffle=True):
    data_size = len(data[0])
    indices = np.arange(data_size)
    if shuffle:
        np.random.shuffle(indices)
    for minibatch_start in np.arange(0, data_size, minibatch_size):
        minibatch_indices = indices[minibatch_start:minibatch_start + minibatch_size]
        yield [_minibatch(d, minibatch_indices) for d in data]

def _minibatch(data, minibatch_idx):
    return data[minibatch_idx] if type(data) is np.ndarray else [data[i] for i in minibatch_idx]

def minibatches(data, batch_size):
    x = np.array([d[0] for d in data])
    y = np.array([d[2] for d in data])
    one_hot = np.zeros((y.size, 3))
    one_hot[np.arange(y.size), y] = 1
    return get_minibatches([x, one_hot], batch_size)

Ablation Study

In [44]:
for ex in train_set:
    print(ex)
   

{'word': [5156, 91, 113, 806, 562, 660, 88, 96, 85, 2131, 97, 109, 1001, 93, 2132, 2133, 144, 96, 2134, 2135, 361, 85, 2136, 91, 2137, 1361, 86, 97, 2138, 199, 2139, 145, 86, 85, 807, 88, 2140, 86, 808, 104, 2141, 2142, 86, 103, 1362, 1363, 89, 2143, 1364, 87], 'pos': [84, 40, 41, 42, 49, 39, 40, 61, 41, 39, 62, 40, 42, 60, 42, 42, 71, 61, 53, 44, 50, 41, 39, 40, 42, 42, 45, 62, 39, 51, 44, 72, 45, 41, 39, 40, 42, 45, 53, 40, 42, 42, 45, 48, 47, 53, 52, 42, 42, 46], 'head': [-1, 5, 5, 5, 5, 45, 9, 9, 9, 5, 9, 15, 15, 12, 15, 9, 20, 20, 19, 20, 5, 22, 20, 25, 25, 20, 20, 20, 20, 28, 28, 20, 45, 34, 45, 36, 34, 34, 34, 41, 41, 38, 34, 45, 45, 0, 48, 48, 45, 45], 'dep': [-1, 11, 15, 6, 37, 36, 11, 17, 15, 36, 17, 11, 14, 11, 6, 36, 17, 17, 25, 3, 29, 15, 1, 11, 6, 36, 17, 17, 29, 18, 12, 17, 17, 15, 7, 11, 36, 17, 33, 11, 6, 36, 17, 22, 30, 0, 11, 6, 36, 17]}
{'word': [5156, 304, 1364, 1002, 2144, 87], 'pos': [84, 42, 42, 55, 42, 46], 'head': [-1, 2, 3, 0, 3, 3], 'dep': [-1, 6, 3, 0, 1, 1

Let us modify the class and test it again.

Case 1: Recall that we have 18 word + 18 pos + 12 dep features

In [46]:
train_set, dev_set, test_set = load_data()
start = time.time()
parser = Parser(train_set)




1. Loading data...
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 2.66 seconds


Before numericalization

In [47]:
print(train_set[1]["word"])
print(train_set[1]["pos"])
print(train_set[1]["head"])
print(train_set[1]["dep"])

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

['ms.', 'haag', 'plays', 'elianti', '.']
['NNP', 'NNP', 'VBZ', 'NNP', '.']
[2, 3, 0, 3, 3]
['compound', 'nsubj', 'root', 'dobj', 'punct']


After numericalization

In [48]:
print(train_set[1]["word"])
print(train_set[1]["pos"])
print(train_set[1]["head"])
print(train_set[1]["dep"])
start = time.time()
train_examples = parser.create_instances(train_set)
print(train_examples[0])

[5156, 304, 1364, 1002, 2144, 87]
[84, 42, 42, 55, 42, 46]
[-1, 2, 3, 0, 3, 3]
[-1, 6, 3, 0, 1, 17]
([5155, 5155, 5156, 91, 113, 806, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 83, 83, 84, 40, 41, 42, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38], [0, 0, 1], 2)


Case2: Try to delete only the 12 dep features and check UAS


In [49]:
train_set, dev_set, test_set = load_data()
start = time.time()
parser = Parser(train_set)




1. Loading data...
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 2.57 seconds


Before numericalization

In [50]:
print(train_set[1]["word"])
print(train_set[1]["pos"])
print(train_set[1]["head"])
print(train_set[1]["dep"])

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

['ms.', 'haag', 'plays', 'elianti', '.']
['NNP', 'NNP', 'VBZ', 'NNP', '.']
[2, 3, 0, 3, 3]
['compound', 'nsubj', 'root', 'dobj', 'punct']


After Numericalization

In [51]:
print(train_set[1]["head"])
print(train_set[1]["dep"])
start = time.time()
train_examples = parser.create_instances(train_set)
print(train_examples[0])

[-1, 2, 3, 0, 3, 3]
[-1, 6, 3, 0, 1, 17]
([5155, 5155, 5156, 91, 113, 806, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 83, 83, 84, 40, 41, 42, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38], [0, 0, 1], 2)


3. Try to delete only the 18 POS features and check UAS

In [52]:
train_set, dev_set, test_set = load_data()
start = time.time()
parser = Parser(train_set)

1. Loading data...
Example 1:  {'word': ['ms.', 'haag', 'plays', 'elianti', '.'], 'pos': ['NNP', 'NNP', 'VBZ', 'NNP', '.'], 'head': [2, 3, 0, 3, 3], 'dep': ['compound', 'nsubj', 'root', 'dobj', 'punct']}
took 2.41 seconds


Before numericalization

In [53]:
print(train_set[1]["word"])
print(train_set[1]["pos"])
print(train_set[1]["head"])
print(train_set[1]["dep"])

train_set = parser.numericalize(train_set)
dev_set   = parser.numericalize(dev_set)
test_set  = parser.numericalize(test_set)

['ms.', 'haag', 'plays', 'elianti', '.']
['NNP', 'NNP', 'VBZ', 'NNP', '.']
[2, 3, 0, 3, 3]
['compound', 'nsubj', 'root', 'dobj', 'punct']


After numericalization

In [56]:
print(train_set[1]["pos"])
print(train_set[1]["head"])

start = time.time()
train_examples = parser.create_instances(train_set)
print(train_examples[0])

[84, 42, 42, 55, 42, 46]
[-1, 2, 3, 0, 3, 3]
([5155, 5155, 5156, 91, 113, 806, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 5155, 83, 83, 84, 40, 41, 42, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 83, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38], [0, 0, 1], 2)


In [57]:
def train(parser, train_data, dev_data, output_path, batch_size=1024, n_epochs=10, lr=0.0005):
    
    best_dev_UAS = 0
    
    optimizer = optim.Adam(parser.model.parameters(), lr=0.001)
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(n_epochs):
        print("Epoch {:} out of {:}".format(epoch + 1, n_epochs))
        dev_UAS = train_for_epoch(
            parser, train_data, dev_data, optimizer, loss_func, batch_size)
        if dev_UAS > best_dev_UAS:
            best_dev_UAS = dev_UAS
            print("New best dev UAS! Saving model.")
            torch.save(parser.model.state_dict(), output_path)
        print("")


def train_for_epoch(parser, train_data, dev_data, optimizer, loss_func, batch_size):
    
    parser.model.train()  # Places model in "train" mode, i.e. apply dropout layer
    n_minibatches = math.ceil(len(train_data) / batch_size)
    loss_meter = AverageMeter()

    with tqdm(total=(n_minibatches)) as prog:
        for i, (train_x, train_y) in enumerate(minibatches(train_data, batch_size)):
            
            #train_x:  batch_size, n_features
            #train_y:  batch_size, target(=3)
            
            optimizer.zero_grad() 
            loss = 0.
            train_x = torch.from_numpy(train_x).long()  #long() for int so embedding works....
            train_y = torch.from_numpy(train_y.nonzero()[1]).long()  #get the index with 1 because torch expects label to be single integer

            # Forward pass: compute predicted logits.
            logits = parser.model(train_x)
            # Compute loss
            loss = loss_func(logits, train_y)
            # Compute gradients of the loss w.r.t model parameters.
            loss.backward()
            # Take step with optimizer.
            optimizer.step()

            prog.update(1)
            loss_meter.update(loss.item())

    print("Average Train Loss: {}".format(loss_meter.avg))
    print("Evaluating on dev set",)
    parser.model.eval()  # Places model in "eval" mode, i.e. don't apply dropout layer
        
    dev_UAS, _ = parser.parse(dev_data)
    print("- dev UAS: {:.2f}".format(dev_UAS * 100.0))
    return dev_UAS

In [64]:
output_dir = "output/{:%Y%m%d_%H%M%S}/".format(datetime.now())
output_path = output_dir + "model.weights"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print(80 * "=")
print("TRAINING")
print(80 * "=")
    
model = ParserModel(embeddings_matrix)
parser.model = model

start = time.time()
train(parser, train_examples, dev_set, output_path,
      batch_size=1024, n_epochs=10, lr=0.0005)